In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

def process_csv_to_2d_array(file_path):
    """
    Processa um arquivo CSV para criar uma matriz bidimensional.
    A primeira coluna será desconsiderada no processamento, mas não removida.
    """
    raw_data = pd.read_csv(file_path, header=None, dtype=str).dropna()
    processed_data = []
    
    for _, row in raw_data.iterrows():
        try:
            processed_row = [eval(cell) if isinstance(cell, str) else cell for cell in row]
            flattened_row = np.hstack([np.array(value) if isinstance(value, list) else np.array([value]) for value in processed_row])
            processed_data.append(flattened_row)
        except Exception as e:
            print(f"Erro ao processar linha: {row.tolist()}. Erro: {e}")
            continue

    data_array = np.array(processed_data)
    return data_array[:, 1:]  # Ignorar a primeira coluna

def reconstruct_iq_signals(i_files, q_files):
    """
    Reconstrói sinais complexos a partir de arquivos de componentes I e Q.
    """
    signals_complex = {}
    for idx, (i_file, q_file) in enumerate(zip(i_files, q_files)):
        print(f"Processando arquivos: {i_file} e {q_file}")
        I = process_csv_to_2d_array(i_file)
        Q = process_csv_to_2d_array(q_file)
        
        # Garantir alinhamento dimensional
        min_rows = min(I.shape[0], Q.shape[0])
        min_cols = min(I.shape[1], Q.shape[1])
        I_aligned = I[:min_rows, :min_cols]
        Q_aligned = Q[:min_rows, :min_cols]
        
        # Reconstruir o sinal complexo
        signals_complex[f"data_{idx}"] = I_aligned + 1j * Q_aligned
    return signals_complex

def apply_mimo_channel(signal, num_tx=3, num_rx=3):
    """
    Aplica um canal MIMO ao sinal de entrada.
    """
    channel_matrix = np.random.randn(num_rx, num_tx) + 1j * np.random.randn(num_rx, num_tx)
    mimo_output = np.zeros((*signal.shape, num_tx * num_rx), dtype=complex)

    idx = 0
    for tx in range(num_tx):
        for rx in range(num_rx):
            mimo_output[:, :, idx] = signal * channel_matrix[rx, tx]
            idx += 1

    return mimo_output

def apply_cfar(data, threshold=1.5):
    """
    Aplica o filtro CFAR para melhorar o contraste do heatmap e reduzir ruídos.
    """
    mean_noise = np.mean(data)
    return np.where(data > mean_noise * threshold, data, 0)

def apply_fft_3d(data, fft_shape):
    """
    Aplica FFT 3D ao longo das dimensões Range, Doppler e Ângulo (MIMO).
    """
    fft_output = np.fft.fftshift(
        np.fft.fftn(
            data,  # Dados de entrada
            s=fft_shape,  # Zero-padding para alcançar a forma desejada
            axes=(0, 1, 2)  # FFT ao longo de Range, Doppler e Ângulo (MIMO)
        ),
        axes=(0, 1, 2)  # Centralizar frequências
    )
    return np.abs(fft_output)  # Retornar a magnitude

def generate_heatmaps(i_files, q_files, num_tx=3, num_rx=3):
    """
    Pipeline completo para processar os sinais IQ, aplicar MIMO e gerar heatmaps FFT conforme o artigo.
    """
    # Reconstruir sinais
    signals_complex = reconstruct_iq_signals(i_files, q_files)
    
    # Aplicar MIMO
    mimo_signal = apply_mimo_channel(signals_complex["data_0"], num_tx=num_tx, num_rx=num_rx)
    
    # Aplicar FFT 3D
    fft_output_3d = apply_fft_3d(mimo_signal, fft_shape=(256, 128, 9))
    
    # Integrar ao longo da dimensão Doppler para melhorar SNR
    hm_hori = np.mean(fft_output_3d, axis=1)  # Range vs. Azimuth
    hm_vert = np.mean(fft_output_3d, axis=2)  # Range vs. Elevation
    
    # Aplicar CFAR
    hm_hori = apply_cfar(hm_hori)
    hm_vert = apply_cfar(hm_vert)
    
    # Suavizar os heatmaps
    hm_hori = gaussian_filter(hm_hori, sigma=1)
    hm_vert = gaussian_filter(hm_vert, sigma=1)
    
    return hm_hori, hm_vert

# Lista de arquivos de exemplo
i_files = ["data_2_I.csv"]
q_files = ["data_2_Q.csv"]

# Gerar os heatmaps
hm_hori, hm_vert = generate_heatmaps(i_files, q_files)

# Carrega o arquivo .npz para comparação
npz_file = "00894_radar.npz"
npz_data = np.load(npz_file)

hm_hori_npz = npz_data['hm_hori']
hm_vert_npz = npz_data['hm_vert']
hm_vert_npz_fixed = np.where(np.isnan(hm_vert_npz), np.nanmean(hm_vert_npz), hm_vert_npz)

# Normaliza
def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

hm_hori_generated_norm = normalize_data(hm_hori)
hm_vert_generated_norm = normalize_data(hm_vert)
hm_hori_npz_norm = normalize_data(hm_hori_npz)
hm_vert_npz_fixed_norm = normalize_data(hm_vert_npz_fixed)

# Plota comparações
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
plt.imshow(hm_hori_generated_norm, cmap='jet', aspect='auto')
plt.colorbar()
plt.title("Heatmap Horizontal Gerado")

plt.subplot(2, 2, 2)
plt.imshow(hm_hori_npz_norm, cmap='jet', aspect='auto')
plt.colorbar()
plt.title("Heatmap Horizontal do .npz")

plt.subplot(2, 2, 3)
plt.imshow(hm_vert_generated_norm, cmap='jet', aspect='auto')
plt.colorbar()
plt.title("Heatmap Vertical Gerado")

plt.subplot(2, 2, 4)
plt.imshow(hm_vert_npz_fixed_norm, cmap='jet', aspect='auto')
plt.colorbar()
plt.title("Heatmap Vertical do .npz")

plt.tight_layout()
plt.show()